# Main

In [ ]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
import re
from string import digits
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut


# # To obtain email address
def get_email(company_suffix):
    url        = f"https://www.baumesse.de{company_suffix}"
    email_html = requests.get(url).text
    email      = BeautifulSoup(email_html, 'html.parser').find('a', string = re.compile(r'\S+@\S+')).text
    return       email

def get_country(query, attempt=1, max_attempts=5):
# Попробовать geopy
    geolocator = Nominatim(user_agent="geoapiForTest")
    zipcode1 = query
    try:
        location = geolocator.geocode(zipcode1).address
        country = str(location).split()[-1]
        return country
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_country(query, attempt=attempt+1)
        raise
        
def get_address(address):
    
    dd = address.find_next_sibling().decode()
    full_address = dd.replace('<br/>', ' ').replace('<dd>', '').replace('</dd>', '').lstrip(' ')
    new_soup = BeautifulSoup(dd, 'html.parser')
    zip_and_city = new_soup.br.next_sibling.text
#     res = any(char.isdigit() for char in address_string)

    try: # full address with street, zip code and city
        return get_country(full_address)
        
        
    except Exception:
        
        try: # only zip code and city
            return get_country(zip_and_city)
            
        except Exception: 
            
            try: # only street and city
                street_and_city = ''.join([i for i in full_address if not i.isdigit()])
                return get_country(street_and_city)
                
            except Exception:
                
                try: # only city
                    city = ''.join([i for i in zip_and_city if not i.isdigit()])
                    return get_country(city)
                    
                except Exception:
                    return "Not correct address"
                    pass
        
def check_if_exists(company):
    try:
        file = pd.read_excel(OUTPUT_PATH)
        if company in file['company'].unique():
            return False
        else: return True   
    except Exception:
        return

def write_to_excel(companies_list):
    # # Exporting into .xlsx file
    df = pd.DataFrame(companies_list)
    if FILE_EXISTENCE:
        with pd.ExcelWriter(OUTPUT_PATH,mode='a', if_sheet_exists='overlay') as writer:  
            df.to_excel(writer, index=False, header=False, startrow=writer.sheets['Sheet1'].max_row)
    else:
        write_to_output_excel = df.to_excel(OUTPUT_PATH, index=False)
        write_to_output_excel 
    print(len(companies_list), f"entries have been added to {OUTPUT_PATH}.") 
      
    
# # Change to your own path where you would like to save the .xlsx file
OUTPUT_PATH    = r'C:\Users\DS Ilya\Desktop\export_dataframe.xlsx'
FILE_EXISTENCE = os.path.exists(OUTPUT_PATH)
# # link - Put URL here
link           = 'https://www.baumesse.de/ausstellersuche/?exhibitor_search_name=&exhibitor_search_tradefair=&exhibitor_search_year=2023&exhibitor_search_product_group=&exhibitor_search_submit=Seek'
response       = requests.get(link)
print(response)

html           = response.text
soup           = BeautifulSoup(html, 'html.parser')
companies      = soup.find_all('h3')
companies_list = []

for company in range(len(companies)):
    
    # Progress count
    print(company+1, "out of", len(companies))
    
    try:
        if FILE_EXISTENCE:
            if check_if_exists(companies[company].text) != True:
                continue
        else: pass
    except Exception:
        print(Exception)
        if FILE_EXISTENCE and len(pd.read_excel(OUTPUT_PATH)) > 0:
            write_to_excel(companies_list)
        pass

    data_dict              = {}
    company_url_suffix     = companies[company].a.get('href')
    address_tag            = companies[company].find_next_sibling().find_next_sibling().dt

    data_dict['company']   = companies[company].text
    data_dict['email']     = get_email(company_url_suffix)
    data_dict['country']   = get_country(get_address(address_tag))
    
    # Getting company name,email and country into a list
    companies_list.append(data_dict)
    
    if company == 10: break
    else: pass
    
print("Done")
write_to_excel(companies_list)
    
   